<a href="https://colab.research.google.com/github/tcae/FollowUpward/blob/master/cpc_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [60]:
%env PYTHONPATH=/env/python:/content/FollowUpward
%env PYTHONPATH
#%env PYTHONPATH=$PYTHONPATH:FollowUpward
#%env PYTHONPATH
!ls /content -l
!ls /content/FollowUpward -l


env: PYTHONPATH=/env/python:/content/FollowUpward
total 8
drwxr-xr-x 5 root root 4096 Mar  2 20:14 FollowUpward
drwxr-xr-x 1 root root 4096 Feb 26 17:33 sample_data
total 772
-rw-r--r-- 1 root root 215503 Mar  2 20:14 Catalyst2PandasData.ipynb
-rw-r--r-- 1 root root   2602 Mar  2 20:14 catalyst_frame.py
-rw-r--r-- 1 root root   9050 Mar  2 20:14 ClassifyTimeSpecific.py
-rw-r--r-- 1 root root   8968 Mar  2 20:14 colab_lc.py
-rw-r--r-- 1 root root   9624 Mar  2 20:14 colab_lin_class.py
-rw-r--r-- 1 root root 303142 Mar  2 20:14 colab_logistic_regression.ipynb
-rw-r--r-- 1 root root   8195 Mar  2 20:14 cpc_classifier.py
-rw-r--r-- 1 root root  11701 Mar  2 20:14 cpc_linear.ipynb
-rw-r--r-- 1 root root   1514 Mar  2 20:14 currency_pairs_2_classifier.py
-rw-r--r-- 1 root root    843 Mar  2 20:14 currency_pairs_mod.py
-rw-r--r-- 1 root root  73461 Mar  2 20:14 logistic_regression.ipynb
-rw-r--r-- 1 root root  30375 Mar  2 20:14 logistic_regression-tcae.ipynb
drwxr-xr-x 2 root root   4096 Mar

In [11]:
!ls FollowUpward






Catalyst2PandasData.ipynb	logistic_regression.ipynb
catalyst_frame.py		__pycache__
ClassifyTimeSpecific.py		README.md
cpc_classifier.py		snippets.py
cpc_linear.ipynb		targets_features.py
currency_pairs_2_classifier.py	TestSnippets.ipynb
currency_pairs_mod.py		test_targets_features.py
df-test.pydata


In [3]:
!pwd


/content


In [26]:
!printenv PYTHONPATH


/env/python:FollowUpward


In [59]:
%cd /content
!pwd
!rm -R FollowUpward
!git clone https://github.com/tcae/FollowUpward.git

/content
/content
rm: cannot remove 'FollowUpward': No such file or directory
Cloning into 'FollowUpward'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 139 (delta 80), reused 83 (delta 40), pack-reused 0
Receiving objects: 100% (139/139), 14.06 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [11]:
!git fetch FollowUpward.git


fatal: not a git repository (or any of the parent directories): .git


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append("./FollowUpward")


In [62]:
#from importlib.machinery import SourceFileLoader
#somemodule = SourceFileLoader('cpc_classifier', 'FollowUpward/cpc_classifier.py').load_module()
#somemodule = SourceFileLoader('TfVectors', 'FollowUpward/targets_features.py').load_module()
# from cpc_classifier import execute
%env PYTHONPATH
%cd /content/FollowUpward
!ls
%env PYTHONPATH

import targets_features
import cpc_classifier
cpc_classifier.execute()

/content/FollowUpward
Catalyst2PandasData.ipynb	 logistic_regression.ipynb
catalyst_frame.py		 logistic_regression-tcae.ipynb
ClassifyTimeSpecific.py		 __pycache__
colab_lc.py			 README.md
colab_lin_class.py		 snippets.py
colab_logistic_regression.ipynb  targets_features.py
cpc_classifier.py		 TestSnippets.ipynb
cpc_linear.ipynb		 test_targets_features.py
currency_pairs_2_classifier.py	 tst_usdt.pydata
currency_pairs_mod.py


AttributeError: ignored

In [25]:
!tail targets_features.py


        return seq



#if __name__ == "__main__":
#    import sys
#    fib(int(sys.argv[1]))
#    currency_data = FeaturesTargets()
#    print(currency_data.performances())



In [45]:
!head cpc_classifier.py -n 30



#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 20 18:40:09 2019

@author: tc

"""

from __future__ import print_function

import os
# import pandas as pd
from targets_features import TfVectors

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


In [30]:
%env PYTHONPATH

'/env/python:/content/FollowUpward'